## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-03-08-49-11 +0000,nypost,India to maintain Russian oil import despite T...,https://nypost.com/2025/08/03/world-news/india...
1,2025-08-03-08-38-44 +0000,bbc,Family condemns Hamas video showing emaciated ...,https://www.bbc.com/news/articles/cpwyxv9nv2vo...
2,2025-08-03-08-11-14 +0000,nypost,Texas Roadhouse hit with civil rights complain...,https://nypost.com/2025/08/03/business/texas-r...
3,2025-08-03-08-09-29 +0000,bbc,Tens of thousands turn out for Sydney pro-Pale...,https://www.bbc.com/news/articles/clydnvw1dg5o...
4,2025-08-03-08-00-00 +0000,cbc,Carney wants to spend an extra $9B on defence ...,https://www.cbc.ca/news/politics/carney-defenc...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2301/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,trump,19
2,russian,7
118,former,7
68,fire,7
3,oil,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
56,2025-08-02-22-28-00 +0000,wsj,A watchdog agency opened an investigation into...,https://www.wsj.com/politics/policy/watchdog-a...,66
6,2025-08-03-07-39-06 +0000,nypost,US envoy tells Israeli hostage families he’s w...,https://nypost.com/2025/08/03/world-news/us-en...,48
105,2025-08-02-15-31-46 +0000,nypost,Trump demurs on pardoning disgraced former Rep...,https://nypost.com/2025/08/02/us-news/presiden...,44
41,2025-08-03-00-48-00 +0000,wsj,Erika McEntarfer led the Bureau of Labor Stati...,https://www.wsj.com/politics/policy/the-econom...,43
45,2025-08-03-00-09-55 +0000,cbc,U.S. watchdog launches investigation into form...,https://www.cbc.ca/news/world/jack-smith-speci...,42


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
56,66,2025-08-02-22-28-00 +0000,wsj,A watchdog agency opened an investigation into...,https://www.wsj.com/politics/policy/watchdog-a...
6,42,2025-08-03-07-39-06 +0000,nypost,US envoy tells Israeli hostage families he’s w...,https://nypost.com/2025/08/03/world-news/us-en...
8,36,2025-08-03-07-21-03 +0000,bbc,Ukraine drone attack sparks fire at Sochi oil ...,https://www.bbc.com/news/articles/ckglyv396ppo...
46,36,2025-08-03-00-04-44 +0000,nypost,Cincinnati police chief under intense scrutiny...,https://nypost.com/2025/08/02/us-news/cincinna...
74,31,2025-08-02-20-20-05 +0000,nypost,Bill Maher slams woke mob over Sydney Sweeney ...,https://nypost.com/2025/08/02/us-news/bill-mah...
99,24,2025-08-02-16-20-00 +0000,wsj,Manhunt Under Way After Four Killed in Montana...,https://www.wsj.com/us-news/four-people-killed...
27,23,2025-08-03-03-00-00 +0000,wsj,Swiss business officials struggle to wrap thei...,https://www.wsj.com/economy/trade/trumps-slap-...
121,21,2025-08-02-12-07-57 +0000,nypost,Curtis Sliwa vows to let voters decide if NYC ...,https://nypost.com/2025/08/02/us-news/sliwa-il...
87,20,2025-08-02-18-39-45 +0000,nypost,Islamic Republic appoints ‘Kennedy of Iran’ to...,https://nypost.com/2025/08/02/world-news/islam...
58,20,2025-08-02-22-15-38 +0000,nypost,Theranos fraudster Elizabeth Holmes jogs at ‘C...,https://nypost.com/2025/08/02/us-news/elizabet...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
